In [ ]:
# -*- coding: utf-8 -*-
"""
Name: Shane Quinn
Student Number: R00144107
Email: shane.quinn1@mycit.ie
Course: MSc Artificial Intelligence
Module: Deep Learning
Date: 01/05/2021
"""

# from google.colab import drive
# drive.mount('/content/gdrive')

# !unzip "/content/gdrive/My Drive/datasets/earth_data.zip" -d "./"

# !ls

import numpy as np
import h5py
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import load_model
from sklearn.metrics import accuracy_score
import functools
import time
import os


def exec_time(func):
    """
    Generic Execution time recorder, pass in function. Records execution time using decorators
    Have used this in previous assignments to record execution time

    Parameters
    ----------
    func : FUNCTION
        Function we're recording and printing execution time of.
    """
    
    @functools.wraps(func)
    def record_exec_time(*args, **kwargs):
        start_time = time.perf_counter()
        mn = func(*args, **kwargs)
        execution_time = time.perf_counter() - start_time
        print("Execution Time: ", execution_time)
        return mn

    return record_exec_time



def part_a_1_basic3():
    """
    6 Convolutional & Pooling layers -> 1 Densely Connected layers -> SoftMax layer

    Returns
    -------
    model : Model object
        Model described above.

    """
    
    input = keras.Input(shape=(64,64,3))
    conv1_op = keras.layers.Conv2D(16, kernel_size=3, activation='relu', padding="same")(input)
    pool1_op = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1_op)
    conv2_op = keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding="same")(pool1_op)
    pool2_op = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2_op)    
    conv3_op = keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding="same")(pool2_op)
    pool3_op = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3_op)
    conv4_op = keras.layers.Conv2D(128, kernel_size=3, activation='relu', padding="same")(pool3_op)
    pool4_op = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4_op)
    conv5_op = keras.layers.Conv2D(86, kernel_size=3, activation='relu', padding="same")(pool4_op)
    pool5_op = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv5_op)
    conv6_op = keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding="same")(pool5_op)
    pool6_op = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv6_op)    
    flat_op = keras.layers.Flatten()(pool6_op)
    dense_op = keras.layers.Dense(512, activation='relu')(flat_op)
    smax_op = keras.layers.Dense(9, activation=tf.nn.softmax)(dense_op)
    model = keras.Model(inputs=input, outputs=smax_op)
    
    return model



def part_a_1_basic5():
    """
    Convolutional & 4 Pooling layers -> 1 Densely Connected layers -> SoftMax layer

    Returns
    -------
    model : Model object
        Model described above.

    """
     
    input = keras.Input(shape=(64,64,3))
    conv1_op = keras.layers.Conv2D(128, kernel_size=3, activation='relu', padding="same")(input)
    pool1_op = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1_op)
    conv2_op = keras.layers.Conv2D(128, kernel_size=3, activation='relu', padding="same")(pool1_op)
    pool2_op = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2_op)    
    conv3_op = keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding="same")(pool2_op)
    conv4_op = keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding="same")(conv3_op)
    pool3_op = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4_op)
    conv5_op = keras.layers.Conv2D(16, kernel_size=3, activation='relu', padding="same")(pool3_op)
    pool4_op = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv5_op)
    
    flat_op = keras.layers.Flatten()(pool4_op)
    dense_op = keras.layers.Dense(512, activation='relu')(flat_op)
    smax_op = keras.layers.Dense(9, activation=tf.nn.softmax)(dense_op)
    model = keras.Model(inputs=input, outputs=smax_op)

    return model
    

def part_a_1_data_augmentation(X, y):
    """
    Take in training data and target class labels and return image generator object

    Parameters
    ----------
    X : Numpy Array
        Training data.
    y : Numpy Array
        Training target classes.
        
    Returns
    -------
    train_generator : Training generator object
        Feed data augmentated images to NN.

    """
    
    train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(shear_range=0.1,
                                                                     zoom_range=0.3,
                                                                     rotation_range=20,
                                                                     horizontal_flip=True, 
                                                                     vertical_flip=True)
    
    train_generator = train_data_gen.flow(X, y, batch_size = 32)
    
    return train_generator
    


def save_model(model, model_name, X, y, X_val, y_val):
    """
    Compile and fit model and save best weights over NUM_EPOCHS epochs

    Parameters
    ----------
    model : Keras Model
        Model.
    model_name : String
        name to save model as.
    X : Numpy Array
        Training data.
    y : Numpy Array
        Training target classes.
    X_val : Numpy Array
        Test data.
    y_val : Numpy Array
        Test target classes.

    Returns
    -------
    None.

    """
    
    image_gen = part_a_1_data_augmentation(X, y)
    NUM_EPOCHS= 20
    check_name = 'weights.hdf5'
    os.path.isfile(check_name)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(check_name, monitor='val_loss',
                                                    mode='min', save_best_only=True, verbose=1)
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(image_gen, epochs=NUM_EPOCHS,  validation_data=(X_val, y_val), callbacks=[checkpoint])
    model.load_weights(check_name)
    model.save(model_name)
    

def create_models(model_names, X, y, X_val, y_val):
    """
    Create 4 models based on 2 models from PartA_1

    Parameters
    ----------
    model_names : List
        Names models will be saved as.
    X : Numpy Array
        Training data.
    y : Numpy Array
        Training target classes.
    X_val : Numpy Array
        Test data.
    y_val : Numpy Array
        Test target classes.

    Returns
    -------
    None.

    """
    
    model3 = part_a_1_basic3()
    model5 = part_a_1_basic5()
    mname1, mname2, mname3, mname4 = model_names 
    save_model(model3, mname1, X, y, X_val, y_val)
    save_model(model3, mname2, X, y, X_val, y_val)
    save_model(model5, mname3, X, y, X_val, y_val)
    save_model(model5, mname4, X, y, X_val, y_val)
    
        
    

    
def ens_predict(models, X):
    """
    Aggregate predictinos of all models and return result

    Parameters
    ----------
    models : List
        All ensemble models.
    X : Numpy array
        Test data.

    Returns
    -------
    result : Numpy array
        Predictions.

    """
    
    y_pred = [model.predict(X) for model in models]     #Retrieve predictinos for each model and store in y_pred
    y_pred = np.array(y_pred)                           #Convert to numpy array  
    avg = np.divide(np.sum(y_pred, axis = 0), 9)        #Find average
    result = np.argmax(avg, axis=1)                     #Return max value
    
    return result


@exec_time 
def main():
    """
    Ensemble model: Create 4 models (using 2 models from PartA_1)
    Aggregate results 

    Returns
    -------
    None.

    """

    X, y, X_val, y_val = loadDataH5()
    models = []    
    model_names = ('m1', 'm2', 'm3', 'm4')
    mname1, mname2, mname3, mname4 = model_names   
    create_models(model_names, X, y, X_val, y_val)
   
    for m in model_names:
        model = load_model(m)
        results = model.evaluate(X_val, y_val, verbose=0)
        print("Loaded model: {}, Accuracy = {}".format(m, results[1]))  
        models.append(model)

    predictions = ens_predict(models, X_val)  
    print(accuracy_score(predictions, y_val))

    
    
def loadDataH5():
    """
    Extract dataset (supplied in assignment)    
    
    Returns
    -------
    trainX : NUMPY ARRAY
        Training data.
    trainY : NUMPY ARRAY
        Training target class values.
    valX : NUMPY ARRAY
        Test data.
    valY : NUMPY ARRAY
        Test target class values.
    """
    
    with h5py.File('earth_data.h5','r') as hf:
        trainX = np.array(hf.get('trainX'))
        trainY = np.array(hf.get('trainY'))
        valX = np.array(hf.get('valX'))
        valY = np.array(hf.get('valY'))
        # print (trainX.shape,trainY.shape)
        # print (valX.shape,valY.shape)
    return trainX, trainY, valX, valY


if __name__=="__main__":
    main()

Epoch 1/20
176/600 [=======>......................] - ETA: 22s - loss: 1.9608 - accuracy: 0.3116

KeyboardInterrupt: ignored